In [ ]:
# Importing Python and external packages
import os
import sys
import importlib
import json
import csv
from dataclasses import dataclass, field, fields
from collections import namedtuple
from typing import Any
from itertools import compress
from pathlib import Path
import pandas as pd
import numpy as np
import sklearn as sk
from scipy.stats import pearsonr, mannwhitneyu

import matplotlib.pyplot as plt
from  matplotlib import __version__ as plt_version
from scipy import signal, stats
# from array import array
# import datetime as dt
# #mne
# import mne_bids
# import mne


In [ ]:
# check some package versions for documentation and reproducability
print('Python sys', sys.version)
print('pandas', pd.__version__)
print('numpy', np.__version__)
# print('mne_bids', mne_bids.__version__)
# print('mne', mne.__version__)
# print('sci-py', scipy.__version__)
print('sci-kit learn', sk.__version__)
print('matplotlib', plt_version)
## FEB 2022:
# Python sys 3.9.7 (default, Sep 16 2021, 08:50:36) 
# [Clang 10.0.0 ]
# pandas 1.3.4
# numpy 1.20.3
# mne_bids 0.9
# mne 0.24.1
# sci-py 1.7.1
# sci-kit learn 1.0.1

In [ ]:
def get_project_path_in_notebook(
    subfolder: str = '',
):
    """
    Finds path of projectfolder from Notebook.
    Start running this once to correctly find
    other modules/functions
    """
    path = os.getcwd()

    while path[-20:] != 'dyskinesia_neurophys':

        path = os.path.dirname(path)
    
    return path

In [ ]:
# define local storage directories
projectpath = get_project_path_in_notebook()
codepath = os.path.join(projectpath, 'code')
figpath = os.path.join(projectpath, 'figures')
datapath = os.path.join(projectpath, 'data')

In [ ]:
os.chdir(codepath)
# own utility functions
import utils.utils_fileManagement as utilsFiles
import utils.utils_windowing as utilsWindows
from utils.utils_fileManagement import (get_project_path,
                                        load_class_pickle,
                                        save_class_pickle,
                                        mergedData,
                                        correct_acc_class)
# own data preprocessing functions
import lfpecog_preproc.preproc_data_management as dataMng
import lfpecog_preproc.preproc_filters as fltrs
# own data exploration functions
import lfpecog_features.feats_read_proc_data as read_data
import lfpecog_features.feats_spectral_baseline as specBase
import lfpecog_features.feats_spectral_features as spectral
import lfpecog_features.feats_spectral_helpers as specHelp
import lfpecog_features.feats_helper_funcs as ftHelp

import lfpecog_preproc.preproc_import_scores_annotations as importClin
import lfpecog_analysis.import_ephys_results as importResults
# import lfpecog_analysis.get_acc_derivs as accDerivs


import lfpecog_plotting.plotHelpers as plotHelp

In [ ]:
import lfpecog_analysis.get_sweetest_channels as get_sweets

image based contact finding (paused)

In [ ]:
importlib.reload(get_sweets)


sub = '008'
chs = ['LFP_L_01', 'LFP_L_02', 'LFP_L_06','LFP_L_08',]
# chs = ['ECOG_R_01', 'ECOG_R_02',  'ECOG_R_06']  # 'ECOG_R_05',

sandwich = get_sweets.get_sweet_bip_sandwich(sub=sub, ch_names=chs,)



In [ ]:
import lfpecog_preproc.tmsi_poly5reader as tmsiReader
import lfpecog_preproc.preproc_load_raw as loadRaw
import lfpecog_preproc.preproc_get_mne_data as loadData
import lfpecog_preproc.preproc_resample as resample
import lfpecog_preproc.preproc_filters as filters

In [ ]:
def get_download_folder(user='habetsj'):
    path = os.getcwd()
    while not path.endswith('habetsj'):
        path = os.path.dirname(path)

    path = os.path.join(path, 'Downloads')
    return path

In [ ]:
filename = '514IL50_M0S0_Rest_test.Poly5'

path = os.path.join(get_download_folder(), filename)
print(os.path.exists(path))
raw = tmsiReader.Poly5Reader(path)



In [ ]:
sfreq = raw.sample_rate
ch_names = raw.ch_names
raw_data = raw.samples[:, sfreq * 10:-sfreq * 10]

run_duration = (1 / sfreq) * raw_data.shape[1]
run_times = np.arange(0, run_duration, 1 / sfreq)

In [ ]:
dat_arr = raw_data[:16, :]
# correct for unit: microVolt
dat_arr /= 1e6

print(ch_names[:16])


# BandPass-Filtering
dat_arr = filters.bp_filter(data=dat_arr, n_timeRows=0, Fs=sfreq,
          l_freq=2, h_freq=98, method='iir',)

# Notch-Filtering
dat_arr = filters.bp_filter(data=dat_arr, n_timeRows=0, Fs=sfreq,
          l_freq=2, h_freq=98, method='iir',)
dat_arr = filters.notch_filter(data=dat_arr,
                                n_timeRows=0,
                                Fs=sfreq,
                                transBW=10,
                                notchW=4,
                                method='fir',
                                verbose=False,)

# Resampling
# dat_arr, sfreq_new = resample.resample(
#     data=dat_arr,
#     n_timeRows=0,
#     Fs_orig=sfreq,
#     Fs_new=2048,
# )

In [ ]:
# %matplotlib qt

for i_ch, ch in enumerate(ch_names[:8]):

    plt.plot(run_times, dat_arr[i_ch, :], label=ch)

plt.legend(loc='upper left', ncol=3)
plt.show()


In [ ]:
# %matplotlib inline

test PSD

In [ ]:
fig, axes = plt.subplots(2, 1, figsize=(8, 8))

for i, (chname, chdat) in enumerate(zip(ch_names[:16], dat_arr)):

    f, psx = signal.welch(chdat, fs=sfreq, nperseg=sfreq)

    if i < 8: i_ax = 0
    else: i_ax = 1
    axes [i_ax].plot(f, psx, label=chname)

for ax in axes:
    ax.set_xlim(0, 40)
    ax.legend(loc='upper left', ncol=3)
    ax.set_ylabel('PSD')
    ax.set_xlabel('Freq')

plt.show()